In [ ]:
from experiments.src.data import ToyData1
from experiments.src.model import BNNRegressor
from experiments.src.experiment import BasicHMCExperiment, BasicMeanFieldGaussianVIExperiment
from jax import random
import jax.nn
import numpy as onp
import optax

In [ ]:
bnn = BNNRegressor(
    nonlin=jax.nn.silu,
    D_X=2,
    D_Y=1,
    D_H=[16, 16],
    biases=True,
    obs_model="loc_scale",
    prior_scale=onp.sqrt(2),
    prior_type="xavier",
    beta=1.0,
    scale_nonlin=lambda xs: jax.nn.softplus(xs) * 0.2 + 1e-2,
)

In [ ]:
data = ToyData1()

In [ ]:
_ = BasicHMCExperiment(
    bnn,
    data,
    num_samples=400,
    num_warmup=150,
).run(random.PRNGKey(11))

In [ ]:
_ = BasicMeanFieldGaussianVIExperiment(
    bnn,
    data,
    num_samples=1000,
    max_iter=160_000,
    lr_schedule=optax.piecewise_constant_schedule(-0.0001, {100_000: 0.5, 130_000: 0.5})
).run(random.PRNGKey(11))
# Is it training? Try scaling up the network!
# (If stuck, take a look at factory.small)